In [1]:
import plotly.graph_objects as go
import pymesh

%load_ext autoreload
%autoreload 2

In [2]:
mesh = pymesh.load_mesh("data/3dmodels/TAL16.obj")

Mixed triangle and quads in the input file
Converting quads in triangles, face order is not kept!


In [3]:
mesh.num_vertices

19549

In [4]:
mesh.faces

array([[  946,  1032,   944],
       [  945,  1033,   947],
       [  950,  1032,   948],
       ...,
       [18139, 19517, 19547],
       [19518, 18110, 18140],
       [19518, 18140, 19548]], dtype=int32)

In [5]:
# Plot mesh
fig = go.Figure(data=[go.Mesh3d(x=mesh.vertices[:,0],
                                y=mesh.vertices[:,1],
                                z=mesh.vertices[:,2],
                                i=mesh.faces[:,0],
                                j=mesh.faces[:,1],
                                k=mesh.faces[:,2],
                                color='lightpink',
                                opacity=0.50)])
fig.show()